In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# To show all columns and rows data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Path
sourcedir = Path(r"""lucas\Desktop\python_data""")

In [ ]:
#BPI sarm cell/slot offline & online history
bpi_sarm = pd.read_csv('Desktop/python_data/bpi_sarm3.csv', parse_dates=True, low_memory=False, dtype=str)
bpi_sarm

In [ ]:
bpi_sarm.info()

In [ ]:
#PRB sarm cell/slot offline & online history
prb_sarm = pd.read_csv('Desktop/python_data/prb_sarm3.csv', parse_dates=True, low_memory=False, dtype=str)
prb_sarm

In [ ]:
prb_sarm.info()

In [ ]:
#BPI redshift cell/slot test history
bpi_redshift = pd.read_csv('Desktop/python_data/bpi_redshift3.csv', parse_dates=True, low_memory=False, dtype=str)
bpi_redshift

In [ ]:
bpi_redshift.info

In [ ]:
#PRB redshift cell/slot test history
prb_redshift = pd.read_csv('Desktop/python_data/prb_redshift3.csv', parse_dates=True, low_memory=False, dtype=str)
prb_redshift

In [ ]:
prb_redshift.info

In [ ]:
#Concatenate SARM data for same structure parameter
offline_history = pd.concat([bpi_sarm, prb_sarm], ignore_index=True, sort=False)
offline_history

In [ ]:
offline_history.shape

In [ ]:
offline_history.info()

In [ ]:
# Concatenate Redshift data for same structure parameter
cell_history = pd.concat([bpi_redshift, prb_redshift], ignore_index=True, sort=False)
cell_history

In [ ]:
cell_history.shape

In [ ]:
cell_history.info()

In [ ]:
#Merge data : offline_history + cell_history
result = pd.merge(offline_history, 
                  cell_history, 
                  how='left', 
                  left_on=['tester_id', 'col_id', 'row_id'], 
                  right_on=['tester_id', 'col_id', 'row_id'])
result

In [ ]:
result.info()

In [ ]:
result.shape

In [ ]:
#Save result with header to result3.csv to the result directory
result.to_csv('Desktop/python_data/result3.csv', index=None, header=True)

In [ ]:
#Format every row of offline_timestamp,online_timestamp, and end_date_time columns into 
# %Y - Year full version : 2020, %m - Month as a number 01-12 : 09, %d - Day of month 01-31 : 01
# %H - Hour 00-23 : 17, %M - Minute 00-59 : 41, %S - Second 00-59 : 08, %f - Microsecond 000000-999999 : 548513
col_ts = ['offline_timestamp', 'online_timestamp', 'end_date_time']
for col in col_ts:
    result.loc[:,col] = pd.to_datetime(result.loc[:,col], format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
#Add new columns to check time different between cell offline timestamp (offline_timestamp) and drive end test (end_date_time)
result.loc[:, 'time_dif'] = (result.loc[:, 'end_date_time'] - result.loc[:, 'offline_timestamp']).astype('timedelta64[s]')
result.loc[:, 'time_dif_abs'] = (result.loc[:, 'end_date_time'] - result.loc[:, 'offline_timestamp']).astype('timedelta64[s]').abs()

result


In [ ]:
#Save result which is included of new 2 columns of time_dif and time_dif_abs
result.to_csv('Desktop/python_data/result3_timedif.csv', index=None, header=True)

In [ ]:
#Create new DF to store result data which is group by offline_timestamp, online_timestamp, and endtimestamp
df1minstart=result.groupby(['offline_timestamp', 'online_timestamp', 'end_date_time'])['time_dif_abs']

In [ ]:
df1minstart

In [ ]:
#DataFrame.assign() = method to assign new columns to a DF, returning a new object
#.transform() = function call func on self producing a DF with transformed values and that has the same axis length as self

result=result.assign(min=df1minstart.transform(min))
result


In [ ]:
#Check min time different of min and time_dif_abs
result = result[result.loc[:, 'min'] == result.loc[:, 'time_dif_abs']]
result

In [ ]:
#Save result with header to result3.csv to the result directory
result.to_csv('Desktop/python_data/result3_final.csv', index=None, header=True)